In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import sklearn as sk

In [ ]:
# from user side

# location
# severity : low, medium , high
# capacity-need : small, medium , large
# service-need : food and water, shelter , evacuation , medical , fire , workforce , protection
# specialization-need : medical , water rescue , search and rescue , fire responce ,  evacuation , earthquake-responce
# medical-facilities-need : basic , trauma care , surgical facilities ,intensive care units
# supplies and resources : low , high


In [ ]:
# from rescue team side
# location
# severity : low , medium , high
# capacity : small, medium , large
# services : food and water, shelter , evacuation , medical , fire , workforce , protection
# availability : available , full , temprarily unavailable
# specialization : medical , water rescue , search and rescue , fire responce , evacuation ,earthquake-responce
# medical facilities : basic , trauma care , surgical facilities ,intensive care units
# supplies and resources : well-stocked , limited supplies 

In [ ]:
from random import choices , choice , randint , uniform, sample

In [ ]:
sample([1,2,3,4,5,6],k=randint(1,6))

In [ ]:
severity=['low','medium','high']
capacity=['small','medium','high']
services=['food and water','shelter','evacuation','medical','fire','workforce','protection']
availability=['available','full','temprarily unavailable']
specialization=['medical','water rescue' , 'search and rescue' , 'fire responce' , 'evacuation' ,'earthquake-responce']
medical_facility=['basic' , 'trauma care' , 'surgical facilities' ,'intensive care units']
supply_and_resource=['well-stocked' , 'limited supplies']

In [ ]:
severity = ["low", "medium", "high"]
capacity = ["small", "medium", "large"]
services = ["medical care", "food and water", "transportation", "language assistance", "counseling"]
availability = ["available", "almost full", "full", "temporarily unavailable"]
specialization = ["medical", "water rescue", "search and rescue", "fire response", "mental health support"]
medical_facility = ["basic first aid", "trauma care", "surgical facilities", "intensive care units", "pediatric care"]
supply_and_resource = ["well-stocked", "limited supplies", "medical equipment available", "pharmaceuticals available"]
calamities = ["earthquake", "flood", "fire", "hurricane", "tsunami", "pandemic"]


In [ ]:
def generate(severity, capacity, services, availability, specialization, medical_facility, supply_and_resource, calamities):
    loc = {"lat": uniform(6, 36), "long": uniform(68, 98)}
    sev = sample(severity, k=1)
    cap = sample(capacity, k=1)
    serv = sample(services, k=randint(1, len(services)))
    avail = sample(availability, k=1)
    special = sample(specialization, k=randint(1, len(specialization)))
    med = sample(medical_facility, k=randint(1, len(medical_facility)))
    resource = sample(supply_and_resource, k=1)
    calamity_needs = sample(calamities, k=randint(1, len(calamities)))
    
    return loc, sev, cap, serv, avail, special, med, resource, calamity_needs


In [ ]:

data = [generate(severity, capacity, services, availability, specialization, medical_facility, supply_and_resource, calamities) for _ in range(100)]
columns = ["Location", "Severity", "Capacity", "Services", "Availability", "Specialization", "Medical Facility", "Supply and Resource","calamities"]
df = pd.DataFrame(data, columns=columns)


In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
one_hot_encoder=MultiLabelBinarizer()

In [ ]:
df_one_hot=df.copy()

In [ ]:
classes=[]

In [ ]:
for column in columns[1:]:
    label_arr=df[column].tolist()
    one_hot_encoder=MultiLabelBinarizer()
    one_hot=one_hot_encoder.fit_transform(label_arr)
    classes.append(one_hot_encoder.classes_)
    
    ll=[list(i) for i in one_hot_encoder.inverse_transform(one_hot)]
    
    df[column]=ll
    df_one_hot[column]=one_hot.tolist()
    

In [ ]:
severity = classes[0]
capacity = classes[1]
services = classes[2]
availability = classes[3]
specialization = classes[4]
medical_facility = classes[5]
supply_and_resource = classes[6]
calamities = classes[7]


In [ ]:
df_one_hot

In [ ]:
df

In [ ]:
from scipy.spatial.distance import cosine
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
def similarity(user,rescue):
    point1=np.array([user['Location']['lat'],user['Location']['long']])
    point2=np.array([rescue['Location']['lat'],rescue['Location']['long']])
    dist=np.linalg.norm(point1 - point2)
    cos_dist=cosine(user['Location']['lat'],rescue['Location']['lat'])
    cos_dist+=cosine(user['Location']['long'],rescue['Location']['long'])
    sev=cosine(user['Severity'],rescue['Severity'])
    cap=cosine(user['Capacity'],rescue['Capacity'])
    serv=cosine(user['Service'],rescue['Services'])
    spec=cosine(user['Specialization'],rescue['Specialization'])
    med=cosine(user['Medical Facility'],rescue['Medical Facility'])
    suppy_and_rescource=cosine(user['Supply and Resource'],rescue['Supply and Resource'])
    cal=cosine(user['calamities'],rescue['calamities'])
    
    dist=dist/100
    
    similarities = np.array([0.9*dist,0.8*sev,0.5*cap,0.3*serv,0.5*spec,0.2*med,0.2*suppy_and_rescource, 0.4*cal])
    return dist ,similarities 
    

In [ ]:
def encoder(values,labels):
    l=np.zeros(len(labels))
    for i in range(len(labels)):
        if labels[i]  in values:
            l[i]=1
    return l

In [ ]:
def process(user):
    user_={
        'Location':user['Location'],
        'Severity': encoder(user['Severity'],severity),
        'Capacity': encoder(user['Capacity'],capacity),
        'Service':encoder(user['Service'],services),
        'Specialization': encoder(user['Specialization'],specialization),
        'Medical Facility':encoder(user['Medical Facility'],medical_facility),
        'Supply and Resource':encoder(user['Supply and Resource'],supply_and_resource),
        'calamities':encoder(user['calamities'],calamities)
    }
    
    return user_

In [ ]:
def similarity_search(user,df,df_one_hot,k=3):
    dist=[]
    user_=process(user)
    print("*")
    for i in range(len(df)):
        rescue=df_one_hot.iloc[i].to_dict()
        si=similarity(user_,rescue)
        
        dist.append((i,si[0],si[1],np.sum(si[1])))
        
    dist=np.array(dist)
    dist_id=np.argsort(dist[:,-1])   
    dist=dist[dist_id[:k]]
    
    user_df = pd.DataFrame.from_dict({key: [value] for key, value in user.items()})
    top_k1=[]
    for i in dist_id[:k]:
        top_k1.append(df.iloc[i].to_dict())
    top_k2=[]
    for i in dist_id[:k]:
        top_k2.append(df_one_hot.iloc[i].to_dict())
        
    rescue_df=pd.DataFrame(top_k1)
    rescue_df_one_hot=pd.DataFrame(top_k2)

    
    return dist , user_df ,rescue_df,rescue_df_one_hot

In [ ]:
user={
    'Location':{
        'lat':26,
        'long':80
    },
    'Severity': ['high'],
    'Capacity': ['large'],
    'Service':['medical care','food and water'],
    'Specialization': ['medical'],
    'Medical Facility': ['intensive care units'],
    'Supply and Resource':['pharmaceutical available'],
    'calamities': ['pandemic']
}

In [ ]:
dist, user_df, rescue_df,rescue_df_one_hot=similarity_search(user,df,df_one_hot)

In [ ]:
user_df

In [ ]:
rescue_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')

In [ ]:
x=[]
y=[]
for i in range(len(df)):
    x.append(df.iloc[i]['Location']['lat'])
    y.append(df.iloc[i]['Location']['long'])

In [ ]:
plt.scatter(x,y)
plt.scatter(rescue_df.iloc[0]['Location']['lat'],rescue_df.iloc[0]['Location']['long'],color='green')
plt.scatter(rescue_df.iloc[1]['Location']['lat'],rescue_df.iloc[1]['Location']['long'],color='blue')
plt.scatter(rescue_df.iloc[2]['Location']['lat'],rescue_df.iloc[2]['Location']['long'],color='black')
plt.scatter(user['Location']['lat'],user['Location']['long'],color='red')

In [ ]:
scaled_data

In [ ]:
df

In [ ]:
def preprocess_for_upload(data,top_k=3):
    values=np.zeros((8,8))
    values[0][0]=data['Location']['lat']
    values[0][1]=data['Location']['long']
        
    values=encoder(data['Severity'],classes[0],values,1)
    values=encoder(data['Capacity'],classes[1],values,2)
    values=encoder(data['Service'],classes[2],values,3)
    values=encoder(data['Specialization'],classes[4],values,4)
    values=encoder(data['Medical Facility'],classes[5],values,5)
    values=encoder(data['Supply and Resource'],classes[6],values,6)
    values=encoder(data['calamities'],classes[7],values,7)

    return values
    
def encoder(values,labels,array,j):
    for i in range(len(labels)):
        if labels[i]  in values:
            array[j][i]=1
    return l

In [ ]:
import pandas

In [ ]:
from pipeline.preprocessing.data_preprocessing import DataPreprocesser

In [ ]:
from pipeline.prediction.prediction import Prediction

In [ ]:
predictor=Prediction()

In [ ]:
all_data=[]
for i in range(len(df)):
    x=df.iloc[i].to_dict()
    x['id']=i
    all_data.append(x)

In [ ]:
user={
    'id': 1010,
    'Location':{
        'lat':26,
        'long':80
    },
    'Severity': ['medium'],
    'Capacity': ['large'],
    'Services':['medical care','food and water'],
    'Specialization': ['medical'],
    'Medical Facility': ['intensive care units'],
    'Supply and Resource':['pharmaceutical available'],
    'calamities': ['pandemic']
}

In [ ]:
user

In [ ]:
res=predictor.similarity_search(user,all_data)

In [ ]:
res

In [ ]:
user

In [ ]:
from scipy.spatial.distance import jaccard

In [ ]:
jaccard([1,1,1,0,1,1],[1,1,1,0,1,1])

In [ ]:
user

In [ ]:
for i in range(3):
    print(all_data[res[i][0]])
    print()
    print()

In [ ]:
from pipeline.component import Data

In [ ]:
from scipy.spatial.distance import hamming

In [ ]:
predictor.classes

In [ ]:
classes

In [ ]:
predictor.classes